# Drift Diffusion Modeling Workshop - Colab Notebook (Using PyDDM)
---
## Setup
First, we install the `pyddm` package if it's not already available in this Colab environment.

In [ ]:
!pip install pyddm --quiet

---
## Imports
Here are the essential packages we will use.

In [ ]:
import numpy as np
import pyddm
import matplotlib.pyplot as plt

---
# STEP 1: Simulate Synthetic DDM Data
---
In this section, we
- set up a model (with some parameters)
- "solve" this model, meaning that some algorithm figures out how to quickly simulate the drift diffusion process with our parameters
- simulate reaction time data with constant drift, boundary, and noise (we don't specify any value so the default ones will be used, except for drift otherwise it will be zero).

Let's now plot the simulated RTs in a histogram separating correct and incorrect trials, + the accuracy of the model


In [ ]:
m = pyddm.gddm(drift=1, bound=1, nondecision=.1, T_dur=3)
sol = m.solve()
sample_sim = sol.sample(k=5000)
plt.hist(sample_sim.choice_upper, bins=30, alpha=0.6, label="Correct (%.1fpc)"%(100*sample_sim.prob('correct')))
plt.hist(sample_sim.choice_lower, bins=30, alpha=0.6, label="Error")
plt.xlabel("Response Time (s)")
plt.ylabel("Count")
plt.title("Simulated DDM RT Distribution")
plt.legend()
plt.show()

And here is a summary of the model

In [ ]:
m.show()

---
# STEP 2: Fitting a Simple DDM to the Simulated Data
---
Here we create another model *BUT* with unspecified parameters.
In other words, we prepare a "blank" model to be fit on our simulated data.

In [ ]:
model_to_fit = pyddm.gddm(drift="d", noise=1.0, bound="B", nondecision=0.1, starting_position=0,
                          parameters={"d": (0,2), "B": (0.3, 2)}, T_dur=3)
model_to_fit.show()

# Let's now fit the "to-be-fitted" model to the samples!

In [ ]:
model_to_fit.fit(sample_sim, lossfunction=pyddm.LossBIC, verbose=False)
model_to_fit.show()
model_to_fit.get_fit_result()

Let's vizualize!

In [ ]:
import pyddm.plot as ddmplot
ddmplot.model_gui_jupyter(model_to_fit, sample=sample_sim)
#ddmplot.model_gui_jupyter(ddm_model)



One additional thing you might want to do is simulate some data and get them in a csv format locally (on your computer). Here is how to do:

In [ ]:
import pandas as pd
from google.colab import files
rt_data = pd.DataFrame({
    "rt": np.concatenate([sample_sim.choice_upper, sample_sim.choice_lower]),
    "response": [1]*len(sample_sim.choice_upper) + [0]*len(sample_sim.choice_lower)
})

rt_data.to_csv("ddm_sample_data.csv", index=False)
print("Saved example data to ddm_sample_data.csv")
files.download("ddm_sample_data.csv")

---
# STEP 3: using real human data
---

In [ ]:
!wget https://raw.githubusercontent.com/mehdisenoussi/misc_utils/refs/heads/main/sim_dataset/df_all_33pp_efficiencyTask.csv -O my_data.csv

In [ ]:
# let's test things on one participant
import pandas as pd
from pyddm import Sample
df = pd.read_csv("my_data.csv")
df.rt = df.rt/1000

pp_data = df[df['sub']=='15']
# cleaning data
pp_data = pp_data[(pp_data['rt']>.200) & (pp_data['rt']<2.000)]


In [ ]:
plt.hist(pp_data.rt[pp_data.cor==1], bins=30)
plt.hist(pp_data.rt[pp_data.cor==0], bins=30)


Let's create a sample based on our real data and define a model to fit this participant's data

In [ ]:
pp_sample = Sample.from_pandas_dataframe(pp_data, rt_column_name="rt", choice_column_name="cor")
model_to_fit_pp = pyddm.gddm(drift="d", noise=1.0, bound="B", nondecision=0.35, starting_position=0,
                          parameters={"d": (0,4), "B": (0.1, 2)}, T_dur=2)
model_to_fit_pp.fit(pp_sample, lossfunction=pyddm.LossBIC, verbose=False)
#model_to_fit_pp.show()
model_to_fit_pp.get_fit_result()

Let's see if it looks good!

In [ ]:
ddmplot.model_gui_jupyter(model_to_fit_pp, sample=pp_sample)